In [1]:
%load_ext autoreload
%autoreload 2
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import os, json, re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
%load_ext autoreload
%autoreload 2

/opt/conda/envs/wordle/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print( torch.__version__)
print(torch.cuda.is_available())

2.3.0+cu121
True


# Load pretrained model and tokenizer

In [3]:
# Config model + quantization
llama3_8b = "meta-llama/Meta-Llama-3-8B-Instruct"
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

# Load in NF4 to fit 15GB GPU
tokenizer = AutoTokenizer.from_pretrained(llama3_8b, padding_side="right")
model = AutoModelForCausalLM.from_pretrained(llama3_8b, 
                                             quantization_config=nf4_config,
                                             device_map="auto", )

# Set pad token to eos token
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:13<00:00,  3.40s/it]


# Prompt_Helper

### prompts

In [12]:
instructions = """

Wordle is game about guessting a 5 letter word. Here are the instructions:

1.  Each guess must be a valid five-letter word.

2.  The color of the position of the letter will change to show you how close your guess was.

3.  If the position is Green (represented by the letter "G"), the letter is in the word, and it is in the correct spot.

4.  If the position is Yellow (represented by the letter "Y"), the letter is in the word, but it is not in the correct spot.

5.  If the position is White, (represented by the letter "W"), the letter is not in the word.

It is played in turns of Guesses and Feedbacks
Fill in the question marks of the json objects.
Always fill out the "reason" section

Guesses must be stricly a five letter word

Only return the json object, and nothing else.
Only return the json object, and nothing else.
Only return the json object, and nothing else.
Only return the json object, and nothing else.
Only return the json object, and nothing else.
"""

prompt_j1 = """
Example game:
Q:
{
    "turn_type" : "Guess",
    "guess" : "BAGEL",
    "reason" : "an initial guess
    
        In this guess, the order of the letters is in the following order:

        1. B
        2. A
        3. G
        4. E
        5. L "
},

A:
{
    "turn_type": "Feedback",
    "result" : "WWWGW"
    "feedback" : "
        Our guess was: 'BAGEL'

        The feedback is: 'WWWGW'

        The feedback for each letter is:

        1. 'W': this means that the letter at position 1, which is 'B', is not in the word
        2. 'W': this means that the letter at position 2, which is 'A', is not in the word
        3. 'W': this means that the letter at position 3, which is 'G', is not in the word
        4. 'G': this means that the letter at position 4, which is 'E', IS IN the word
        5. 'W': this means that the letter at position 5, which is 'L', is not in the word",
    "next_action" : "guess again"
},

Q: 
{
    "turn_type" : "Guess",
    "guess" : "THREW",
    "reason" : "
        To reiterate: 
        1. The letter 'B' is not in the word at position 1, because the letter (in the feedback at position 1) was 'W'
        2. The letter 'A' is not in the word at position 2, because the letter (in the feedback at position 2) was 'W'
        3. The letter 'G' is not in the word at position 3, because the letter (in the feedback at position 1) was 'W'
        4. The letter 'E' IS IN in the word at posiiton 4, because the letter (in the feedback at position 1) was 'G'
        5. The letter 'L' is not in the word at position 5, because the letter (in the feedback at position 1) was 'W'
    
        Here is the list of invalid letters: [B, A, G, L]

        The next guess is the word "Threw" is a good guess, because:
        1. The letter 'T' at position 1 is in the list of invalid letters.
        2. The letter 'H' is position 2 is in the list of invalid letters.
        3. The letter 'R' at position 3 is in the list of invalid letters.
        4. The letter 'E' at position 4 is not in the list of invalid letters, and is in the correct position.
        5. The letter 'W' at position 5 is in the list of invalid letters. "
}

A:
{
    "turn_type" : "Feedback",
    "result" : "GGGGW"
    "Feedback" : "
        Our guess was: 'THREW'

        The feedback is: 'GGGGW'

        The feedback for each letter is:

        1. 'G': this means that the letter at position 1, which is 'T', IS IN in the word
        2. 'G': this means that the letter at position 2, which is 'H', IS IN in the word
        3. 'G': this means that the letter at position 3, which is 'R', IS IN in the word
        4. 'G': this means that the letter at position 4, which is 'E', IS IN the word
        5. 'W': this means that the letter at position 5, which is 'W', is not in the word "
        "next_action" : "guess again"
} , 

Q:
{ 
    turn_type: "Guess",
    "guess: "THREE", 
    "reason" : 
        "Our previous guess was THREW
        To reiterate: 
        1. The letter 'T' IS IN in the word at position 1, because the letter (in the feedback at position 1) was 'G'
        2. The letter 'H' IS IN in the word at position 2, because the letter (in the feedback at position 2) was 'G'
        3. The letter 'R' IS IN in the word at position 3, because the letter (in the feedback at position 3) was 'G'
        4. The letter 'E' IS IN in the word at posiiton 4, because the letter (in the feedback at position 4) was 'G'
        5. The letter 'W' is not in the word at position 5, because the letter (in the feedback at position 5) was 'W'
    
        Here is the list of invalid letters: [B, A, G, L, W]

        The next guess is the word "THREW" is a good guess, because:
        1. The letter 'T' at position 1 is not in the list of invalid letters, and is in the correct position.
        2. TThe letter 'H' at position 2 is not in the list of invalid letters, and is in the correct position.
        3. The letter 'R' at position 3 is not in the list of invalid letters, and is in the correct position.
        4. The letter 'E' at position 4 is not in the list of invalid letters, and is in the correct position.
        5. The letter 'W' at position 5 is not in the list of invalid letters."
},

A:
{   "turn_type" : "Feedback",
    "result" : "GGGGG"
    "feedback" : "
        Our guess was: 'THREW'

        The feedback is: 'GGGGG'

        The feedback for each letter is:

        1. 'G': this means that the letter at position 1, which is 'T', IS IN the word, and is in the correct position
        2. 'G': this means that the letter at position 2, which is 'H', IS IN the word, and is in the correct position
        3. 'G': this means that the letter at position 3, which is 'R', IS IN the word, and is in the correct position
        4. 'G': this means that the letter at position 4, which is 'E', IS IN the word, and is in the correct position
        5. 'G': this means that the letter at position 5, which is 'E', IS IN the word, and is in the correct position

        Because all letters are in the word and is in the correct position, we guessed the word correctly!",
    "next_action" : "END"
)  


Next Game:
Q:

"""
# {
#     "turn_type" : "Guess",
#     "guess" : "?", 
#     "reason" : "Random initial guess"
# }

In [13]:
from wordle import Wordle

class Prompt_Interface:
    def __init__(self, instructions):
        self.instructions = instructions,


    def feedback(self, guess, response, guess_number):
        # response given from wordle object
        # guess generated by the model
        # guess_number is how many guesses the model has made

#     Feedback:(
    #     Our guess was: 'THREW'

    #     The feedback is: 'GGGGW'

    #     The feedback for each letter is:

    #     1. 'G': this means that the letter at position 1, which is 'T', IS IN in the word
    #     2. 'G': this means that the letter at position 2, which is 'H', IS IN in the word
    #     3. 'G': this means that the letter at position 3, which is 'R', IS IN in the word
    #     4. 'G': this means that the letter at position 4, which is 'E', IS IN the word
    #     5. 'W': this means that the letter at position 5, which is 'W', is not in the word
#       )  
        
        is_in_word = ["IS IN" if response[i] == 'G' else 'not in' for i in range(5)]
        if response == "GGGGG": 
            next_action = "DONE"
            victory_speech = "Because all letters are in the word and is in the correct position, we guessed the word correctly!"
        else:
            next_action = "guess again"
            victory_speech =""

        feedback = f"""
         A: 
         {{ 
            "turn_type": "Feedback",
            "result" : "{response}"    
            "feedback" : 
                Our guess was: '{guess}'

                The response was '{response}'

                The feedback for each letter is: 
                
                1. '{response[0]}': this means that the letter at position 1, which is '{guess[0]}', {is_in_word[0]} the word
                2. '{response[1]}': this means that the letter at position 2, which is '{guess[1]}', {is_in_word[1]} the word
                3. '{response[2]}': this means that the letter at position 3, which is '{guess[2]}', {is_in_word[2]} the word
                4. '{response[3]}': this means that the letter at position 4, which is '{guess[3]}', {is_in_word[3]} the word
                5. '{response[4]}': this means that the letter at position 5, which is '{guess[4]}', {is_in_word[4]} the word
                {victory_speech}",
            "next_action" : "{next_action}"
        }}
        Q: 
        {{
           "turn_type" : "guess",
           "guess" : "?",
           "reason" : "?" 

        }}
        """
        if response == 'GGGGG': 
            return f"Congrats you found the correct word {guess} in {guess_number+1} guesses, because all letters were in the word and in the right position!"

        
        return feedback


    def get_zero_shot_instructions(self):
        message = [
            {"role": "system", "content": self.instructions},
            {"role": "user", "content": "Enter your first guess in a format like COVER"},
        ]
        return message

    def get_few_shot_instructions(self, shots=2):
        shot = 1
        
        message = [{"role": "system", "content": self.instructions},
                   {"role": "user", "content": prompt_j1}]
        # for s in range(shots):
        #     message.append({"role": "system", "content": "Example #1:"})
        #     w = Wordle()
        #     w.set_answer(self.examples[0]['answer'])
        #     for gnum in range(len(self.examples[0]['guesses'])):
        #         guess = self.examples[s]['guesses'][gnum]
        #         message.extend([
        #             {"role": "user", "content": f"Enter guess #{gnum + 1}"},
        #             {"role": "assistant", "content": guess},
        #             {"role": "user", "content": self.feedback(guess, w.turn(guess), gnum)},
        #         ])

        return message
        

# Create Wordle game and prompt

In [14]:
from wordle import Wordle
game = Wordle()
p = Prompt_Interface(instructions)

In [15]:
# Get a few-shot instruction
conversation = p.get_few_shot_instructions(2)
conversation

[{'role': 'system',
  'content': ('\n\nWordle is game about guessting a 5 letter word. Here are the instructions:\n\n1.  Each guess must be a valid five-letter word.\n\n2.  The color of the position of the letter will change to show you how close your guess was.\n\n3.  If the position is Green (represented by the letter "G"), the letter is in the word, and it is in the correct spot.\n\n4.  If the position is Yellow (represented by the letter "Y"), the letter is in the word, but it is not in the correct spot.\n\n5.  If the position is White, (represented by the letter "W"), the letter is not in the word.\n\nIt is played in turns of Guesses and Feedbacks\nFill in the question marks of the json objects.\nGuesses must be stricly a five letter word\n\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\n',)

In [16]:

def extract_guess(json_string):
    # Example
    # {
    #     "turn_type" : "Guess",
    #     "guess" : "?", 
    #     "reason" : "Random initial guess"
    # }
    guess_idx = json_string.find("guess")
    remaining = json_string[guess_idx+len('guess'):]
    end = remaining.find("\n")
    guess = remaining[:end]
    guess = re.sub(r"[^a-zA-Z]+","", guess)
    return guess

In [17]:
test = """
# Example
    # {
    #     "turn_type" : "Guess",
    #     "guess" : "asdf", 
    #     "reason" : "Random initial guess"
    # }
"""
extract_guess(test)


'asdf'

# Simple demo

In [21]:
# reset and init
from wordle import Wordle
game = Wordle()
p = Prompt_Interface(instructions)

# Get a few-shot instruction
conversation = p.get_few_shot_instructions(2)
print(conversation)

i = 0
max_turns = 100
for t in range(max_turns):
    print("Turn:", t)

    input_ids = tokenizer.apply_chat_template(
        conversation,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.4,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    response_str = tokenizer.decode(response, skip_special_tokens=True)

    guess_str = tokenizer.decode(response, skip_special_tokens=True)
    print("\tGuess 1:", guess_str)
    # guess_str = extract_guess(guess_str)
    # print("\tGuess 2:", guess_str)
    try:
        guess_eval = game.turn(guess_str)
        user_response = p.feedback(guess_str, guess_eval, i)
        print("feedback:\n", user_response)
        conversation.extend([
            {'role': 'assistant', 'content': guess_str},
            {'role': 'user', 'content': user_response}
        ])
    except ValueError as e:
        conversation.extend([
            {'role': 'assistant', 'content': guess_str},
            {'role': 'user', 'content': str(e)}
        ])
        
    if guess_eval == 'GGGGG':
        conversation.append({'role': 'system', 'content': f"Correct! The answer is {game.get_answer()}"})
    elif i >= 100:
        conversation.append({'role': 'system', 'content': f"100 guesses exhausted! The correct answer is {game.get_answer()}"})
        break
    else:
        i += 1

[{'role': 'system', 'content': ('\n\nWordle is game about guessting a 5 letter word. Here are the instructions:\n\n1.  Each guess must be a valid five-letter word.\n\n2.  The color of the position of the letter will change to show you how close your guess was.\n\n3.  If the position is Green (represented by the letter "G"), the letter is in the word, and it is in the correct spot.\n\n4.  If the position is Yellow (represented by the letter "Y"), the letter is in the word, but it is not in the correct spot.\n\n5.  If the position is White, (represented by the letter "W"), the letter is not in the word.\n\nIt is played in turns of Guesses and Feedbacks\nFill in the question marks of the json objects.\nGuesses must be stricly a five letter word\n\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\nOnly return the json object, and nothing else.\n',)},

KeyboardInterrupt: 

In [ ]:
test = """
# Example
    # {
    #     "turn_type" : "Guess",
    #     "guess" : "asdf", 
    #     "reason" : "Random initial guess"
    # }
"""
extract_guess(test)


'asdf'

In [ ]:
test = """
# Example
    # {
    #     "turn_type" : "Guess",
    #     "guess" : "asdf", 
    #     "reason" : "Random initial guess"
    # }
"""
extract_guess(test)


'asdf'

In [ ]:
for turn in conversation:
    print(turn)

{'role': 'system', 'content': ('Wordle is a 5 letter word guessing game. Each letter in a guess will either be a correct letter in correct position, an invalid letter or a correct letter present in the word but in a wrong position. After each guess, we get a result',)}
{'role': 'system', 'content': 'Example #1:'}
{'role': 'user', 'content': 'Enter guess #1'}
{'role': 'assistant', 'content': 'BAGEL'}
{'role': 'user', 'content': 'Letter B is not anywhere present in the word, Letter A is not anywhere present in the word, Letter G is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct position, Letter L is not anywhere present in the word, '}
{'role': 'user', 'content': 'Enter guess #2'}
{'role': 'assistant', 'content': 'COVER'}
{'role': 'user', 'content': 'Letter C is not anywhere present in the word, Letter O is not anywhere present in the word, Letter V is not anywhere present in the word, Letter E at Position 3 is in the word and at the correct po